In [10]:
import os
import re
import geopandas as gpd
from lxml import etree as ET
from shapely.geometry import LineString

In [11]:
# Convertir 'Flight Time' de milisegundos a minutos y segundos (MM:SS)
def convertir_tiempo(ms):
    try:
        ms = int(ms.strip())  # Convertir a entero y eliminar espacios extra
        segundos = ms // 1000  # Convertir milisegundos a segundos
        minutos = segundos // 60
        segundos = segundos % 60
        return f"{minutos}:{segundos:02d}"  # Formato MM:SS (ej. 9:35)
    except (ValueError, AttributeError):
        return None  # Si hay error, devolver None

In [12]:
def convertir_kmls_a_shapefile(kml_folder, shapefile_path):
    """
    Convierte varios archivos KML en un solo shapefile con proyección UTM zona 20S.
    Extrae valores de ExtendedData y los agrega como columnas.

    Parámetros:
    kml_folder (str): Carpeta que contiene los archivos KML.
    shapefile_path (str): Ruta donde se guardará el archivo shapefile combinado.
    """
    # Definir namespace de KML
    NAMESPACE = {"kml": "http://www.opengis.net/kml/2.2"}

    # Listas para almacenar los datos
    line_geometries = []
    fechas = []
    horas = []
    ids = []
    
    aircraft_names = []
    flight_controller_ids = []
    pilot_names = []
    flight_times = []
    mode_selections = []
    heights = []
    route_spacings = []
    task_flight_speeds = []
    task_areas = []
    spray_amounts = []

    if not os.path.isdir(kml_folder):
        print(f"❌ Error: La carpeta {kml_folder} no existe.")
        return

    for filename in os.listdir(kml_folder):
        if filename.endswith('.kml'):
            kml_path = os.path.join(kml_folder, filename)

            try:
                with open(kml_path, 'r', encoding='utf-8') as file:
                    tree = ET.parse(file)
                    root = tree.getroot()
            except Exception as e:
                print(f"❌ Error al leer {filename}: {e}")
                continue
            
            # Extraer fecha, hora e ID del nombre del archivo
            match = re.search(r'T\d+_(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})_(R\d+)\.kml', filename)
            if not match:
                print(f"⚠ Advertencia: Nombre de archivo {filename} no coincide con el patrón esperado.")
                continue
            
            date = f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
            time = f"{match.group(4)}:{match.group(5)}:{match.group(6)}"
            record_id = match.group(7)

            # Buscar Placemark en XML
            placemarks = root.findall('.//Placemark')
            if not placemarks:
                placemarks = root.findall('.//kml:Placemark', NAMESPACE)

            if not placemarks:
                print(f"⚠ Advertencia: No se encontraron <Placemark> en {filename}.")
                continue

            for placemark in placemarks:
                # Buscar LineString correctamente
                line_string = placemark.find('.//LineString')
                if line_string is None:
                    line_string = placemark.find('.//kml:LineString', NAMESPACE)
                
                if line_string is not None:
                    # Buscar Coordinates correctamente
                    coordinates = line_string.find('.//coordinates')
                    if coordinates is None:
                        coordinates = line_string.find('.//kml:coordinates', NAMESPACE)

                    if coordinates is not None and coordinates.text.strip():
                        coords = coordinates.text.strip().split()
                        points = [tuple(map(float, coord.split(',')[:2])) for coord in coords]  # Solo lat, lon
                        line_geometries.append(LineString(points))
                        fechas.append(date)
                        horas.append(time)
                        ids.append(record_id)

                        # Extraer datos de ExtendedData correctamente
                        extended_data = placemark.find('.//ExtendedData')
                        if extended_data is None:
                            extended_data = placemark.find('.//kml:ExtendedData', NAMESPACE)
                        
                        data_dict = {}
                        if extended_data is not None:
                            for data in extended_data.findall('.//Data') or extended_data.findall('.//kml:Data', NAMESPACE):
                                if data is None:
                                    continue  # Skip if there's no Data element

                                name = data.get('name')
                                value_element = data.find('.//value') or data.find('.//kml:value', NAMESPACE)

                                value = value_element.text.strip() if value_element is not None and value_element.text else None
                                if name:
                                    data_dict[name] = value  # Store only if the name exists
                    
                        # Agregar valores de ExtendedData a las listas
                        aircraft_names.append(data_dict.get("Aircraft Name", None))
                        flight_controller_ids.append(data_dict.get("Flight Controller ID", None))
                        pilot_names.append(data_dict.get("Pilot Name", None))
                        flight_times.append(convertir_tiempo(data_dict.get("Flight Time", "0")))
                        mode_selections.append(data_dict.get("Mode Selection", None))
                        heights.append(data_dict.get("Height", None))
                        route_spacings.append(data_dict.get("Route Spacing", None))
                        task_flight_speeds.append(data_dict.get("Task Flight Speed", None))
                        task_areas.append(data_dict.get("Task Area", None))
                        spray_amounts.append(data_dict.get("Spray amount", None))

    if not line_geometries:
        print("❌ No se encontraron geometrías válidas en los archivos KML.")
        return

    # Crear un GeoDataFrame con todas las columnas
    gdf = gpd.GeoDataFrame({
        'fecha': fechas,
        'hora': horas,
        'id': ids,
        'aircraft_name': aircraft_names,
        'flight_controller_id': flight_controller_ids,
        'pilot_name': pilot_names,
        'flight_time': flight_times,
        'mode_selection': mode_selections,
        'height': heights,
        'route_spacing': route_spacings,
        'task_flight_speed': task_flight_speeds,
        'task_area': task_areas,
        'spray_amount': spray_amounts,
        'geometry': line_geometries
    }, crs='EPSG:4326')

    # Transformar a UTM zona 20S (EPSG:32720)
    gdf_utm = gdf.to_crs(epsg=32720)

    # Guardar como shapefile
    gdf_utm.to_file(shapefile_path, driver='ESRI Shapefile')

    print(f"✅ Archivo shapefile guardado en: {shapefile_path}")


In [13]:
ruta = 'S10_CAMPODULCE'

In [14]:
# Directorio donde están los archivos KML
path_kml = r'C:\vuelos\\' + ruta
contenido = os.listdir(path_kml)
len(contenido)

# Ejemplo de uso
kml_folder = f'C:/vuelos/{ruta}'
shapefile_path = f'C:/vuelos/{ruta}/res.shp'
convertir_kmls_a_shapefile(kml_folder, shapefile_path)

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_16176\3559005420.py:97: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  value_element = data.find('.//value') or data.find('.//kml:value', NAMESPACE)


✅ Archivo shapefile guardado en: C:/vuelos/S10_CAMPODULCE/res.shp


C:\Users\bismarksr\AppData\Local\Temp\ipykernel_16176\3559005420.py:141: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_utm.to_file(shapefile_path, driver='ESRI Shapefile')


In [6]:
kml_folder

'C:/vuelos/S10_CAMPODULCE'